In [ ]:
#jobs to do
#fix the viewport
#how to integrate the user head position with the stalling mechanism

#PROBLEM

#show one chart of average buffering time of multiple users over baselines
#show one chart of average data transmission over network for our models and 3 baseline
#show one example graph of buffering time of our models and 2 baselines
#show one example graph of data transmission over network for our models and 3 baseline
#show the different between diffrent windows size using stalling time and data transmission metrics

In [3]:
t = 0;


In [4]:
ls trace_simulation_data/


iid0.log       iid16.log      iid29_tm2.log  iid6_sal.log
iid0_sal.log   iid16_pos.log  iid2.log       iid6_tm2.log
iid10.log      iid16_sal.log  iid2_sal.log   iid7.log
iid10_pos.log  iid16_tm2.log  iid2_tm2.log   iid7_pos.log
iid10_sal.log  iid17.log      iid30_tm2.log  iid7_sal.log
iid10_tm2.log  iid17_pos.log  iid31_tm2.log  iid7_tm2.log
iid11.log      iid17_sal.log  iid32_tm2.log  iid8.log
iid11_pos.log  iid17_tm2.log  iid33_tm2.log  iid8_pos.log
iid11_sal.log  iid18.log      iid34_tm2.log  iid8_sal.log
iid11_tm2.log  iid18_pos.log  iid35_tm2.log  iid8_tm2.log
iid12.log      iid18_sal.log  iid36_tm2.log  iid9.log
iid12_pos.log  iid18_tm2.log  iid37_tm2.log  iid9_pos.log
iid12_sal.log  iid19.log      iid38_tm2.log  iid9_sal.log
iid12_tm2.log  iid19_pos.log  iid39_tm2.log  iid9_tm2.log
iid13.log      iid19_sal.log  iid3.log       order_test_list
iid13_pos.log  iid19_tm2.log  iid3_sal.log   test_result
iid13_sal.log  iid1.log       iid3_tm2.log   tilesize_map_alien
iid13_tm2.log  iid

In [5]:
import pickle
test_result = pickle.load(open('trace_simulation_data/test_result'))
order_test_list = pickle.load(open('trace_simulation_data/order_test_list'))



In [6]:
#THIS CODE IS FOR PREDICTOR TO CONVERT TEST_RESULT TO FOV

def salient_to_centertile(_y, _h, _w, _r=3):
    #TODO: given tile probabilities, output mark tiles with highest probability.
    #INPUT: tiles with different values
    #OUTPUT: mark highest value tiles by 1, the rest by -1
    p_y = _y.reshape(_h, _w)

    #select max tile for both y and fx
    tile_y = np.zeros(shape=p_y.shape)-1
    tile_y[p_y==p_y.max()] = 1
    return tile_y

def centertile_to_fov_v0(_tile_y, _r=3):
    #TODO: convert tile with value 1 to fov tiles
    #INPUT: tiles with one center tiles
    
    #create field of view for y and fx
    i, j = np.unravel_index(_tile_y.argmax(), _tile_y.shape)
    _tile_y[max(0, i-_r):i+_r, max(0, j-_r):j+_r] = 1
    
    return _tile_y

##########
def enlarge_rec(_tile_map, _r=3):#create a larger rectangular to make the wrap field of view
    h, w = _tile_map.shape
    temp = np.zeros(shape=(_r*2 + h, _r*2 + w)) + _tile_map.min()
    temp[_r:_r+h, _r:_r+w] = _tile_map
    
    return temp

def shrink_rec(_tile_map, _r=3):
    h, w = _tile_map.shape
    temp = _tile_map[_r:h-_r, _r:w-_r]
    return temp

def wrap_rec(_tile_map, _r=3):
    #TODO: wrap max elements from left to right and right to left, bottom to top and top to bottom
    #in order to preserve the 360 properties
    
    h, w = _tile_map.shape
    #left piece
    _tile_map[_r:h-_r, _r:_r+_r] = np.maximum(_tile_map[_r:h-_r, w-_r:], _tile_map[_r:h-_r, _r:_r+_r])
    #right piece
    _tile_map[_r:h-_r, w-_r-_r:w-_r] = np.maximum(_tile_map[_r:h-_r, :_r], _tile_map[_r:h-_r, w-_r-_r:w-_r])
    #top piece
    _tile_map[_r:_r+_r, _r:w-_r] = np.maximum(_tile_map[h-_r:, _r:w-_r], _tile_map[_r:_r+_r, _r:w-_r])
    #bottom piece
    _tile_map[h-_r-_r:h-_r, _r:w-_r] = np.maximum(_tile_map[:_r, _r:w-_r], _tile_map[h-_r-_r:h-_r, _r:w-_r])
    
    #topleft
    _tile_map[_r:_r+_r, _r:_r+_r] = np.maximum(_tile_map[h-_r:, w-_r:], _tile_map[_r:_r+_r, _r:_r+_r])
    #bottomleft
    _tile_map[h-_r-_r:h-_r, _r:_r+_r] = np.maximum(_tile_map[:_r, w-_r:], _tile_map[h-_r-_r:h-_r, _r:_r+_r])
    return _tile_map

def expand_fov(_tile_map, _r=3, _ex=1):
    h1, w1 = _tile_map.shape
    temp = enlarge_rec(_tile_map, _r)
    temp1 = np.copy(temp)
    h, w = temp.shape
    
    
    for i in range(h):
        for j in range(w):
            m = _tile_map.max()
            if temp1[i, j] != m:
                temp1[i, j] = m if temp[min(h-1, i+1), j] == m or temp[i, min(w-1, j+1)] == m or temp[min(h-1, i+1), min(w-1, j+1)] == m\
                        or temp[max(0, i-1), j] == m or temp[i, max(0, j-1)] == m or temp[max(0, i-1), max(0, j-1)] == m \
                        or temp[max(0, i-1), min(w-1, j+1)] == m or temp[min(h-1, i+1), max(0, j-1)] == m else 0;
    temp2 = np.copy(temp1)
    temp2 = wrap_rec(temp2, _r)
    return shrink_rec(temp2, _r)

def centertile_to_fov(_tile_y, _r=3):
    #TODO: convert tile with value 1 to fov tiles
    #INPUT: tiles with one center tiles
    
    #preprocessing
    temp1 = enlarge_rec(_tile_y, _r)
    
    #create field of view for y and fx
    i, j = np.unravel_index(temp1.argmax(), temp1.shape)
    temp1[max(0, i-_r):i+_r, max(0, j-_r):j+_r] = 1
    
    temp2 = wrap_rec(temp1, _r)
    temp3 = shrink_rec(temp2, _r)
    
    return temp3


###########


def baseline1_current_salient(_X, _time_index):
    #given feature X, with shape (N, look_back, 2, H, W)#(770, 15, 2, 9, 16)
    #return the matrix
    return _X[_time_index][0]

def baseline2_current_headpos(_X, _time_index):
    return _X[_time_index][1]

In [7]:
#this module must answer which tiles to fetch 
#what is input for the predictor: time

                                                                                                                                            

In [8]:
import pickle
import numpy as np
import logging

#logging.basicConfig(level=logging.DEBUG) 
logging.getLogger().setLevel(logging.DEBUG)
#THIS IS DATA STRUCTURE FOR SERVER

def load_tilesize_map(_tag):
    file_path_template = './trace_simulation_data/tilesize_map_{}'
    tilesize_map_filename = file_path_template.format(_tag)
    return pickle.load(open('./trace_simulation_data/tilesize_map_{}'.format(_tag), 'rb'))

def load_tilesize_map_dict():
    H_list = [0,  160,  320,  480,  640,  800,  960, 1120, 1280]
    W_list = [0,  160,  320,  480,  640,  800,  960, 1120, 1280, 1440, 1600,
           1760, 1920, 2080, 2240, 2400]

    topic_dict = {'1': 'skiing.mp4', '0': 'conan1.mp4', '3': 'conan2.mp4', '2': 'alien.mp4', #'5': 'war.mp4',\ 
                  '4': 'surfing.mp4', '7': 'football.mp4', '6': 'cooking.mp4', '8': 'rhinos.mp4'}
    tag_dict = {k:topic_dict[k].replace('.mp4', '') for k in topic_dict}
    tilesize_map_dict = {tag: load_tilesize_map(tag_dict[tag]) for tag in tag_dict}
    return tilesize_map_dict

def server_get_tile_size(_topic_idx, _time, _tile_map):
    #TODO: server side, return the total size given tilemap and the tilesize map dict and time
    if "tilesize_map_dict" not in locals():
        tilesize_map_dict = load_tilesize_map_dict()
        
    time = int(_time/1000)#get the nearest second chunk, simulator use 1000*time
    tilesize_map = tilesize_map_dict[str(_topic_idx)]
        
    tilesize = tilesize_map[:, :, time, :]#this code has not been tested
    tilesize[_tile_map==-1] = 0
    return tilesize #THIS CODE NEED TEST



In [9]:
def clean_buffer(_querry_time):
    #given querry time, clear all stale data, i.e. the item with chunk_timestamp < _queery_time
    
    
    global GBUFFER
    clean_count = 0
    while len(GBUFFER) > 0:
        if _querry_time > GBUFFER[0][2]:
            d = _querry_time - GBUFFER[0][2]
            GBUFFER[0][1] -= d
            GBUFFER[0][2] = _querry_time
            if GBUFFER[0][1] < 0:
                GBUFFER = GBUFFER[1:]
            clean_count += 1
        else:
            break
    #print 'cleaning BUFFER called, do {} times'.format(clean_count)       

    
#WILL NEED TO INCLUDE HEAD POSITION THROUGH TIME FOR THIS
#this buffer consume totally ignore the user fov.
#if taken into account user fov over time (from GTIME to GTIME + _time)
#then stalling will happens at some points
#this function is TESTED and worked as expected
def buffer_consume(_time):
    #consume global buffer based on time passed
    #if a buffer 
    #updated global variable GBUFFER content, GSTALL_LIST flag
    #return stall_time, which is the time system will be stall
    
    global GTIME
    global GSTALLING
    global GBUFFER
    stall_time = 0
    
    clean_buffer(GTIME)
    

    if GSTALLING == True:
        #is stalling mean empty buffer.
        if GBUFFER != []: 
            temp = [item[1] for item in GBUFFER]
            if sum(temp) > tracesim_header.base_buffer_time:
                raise Exception ('Why there is this case in here, why buffer fillup does not fulfill?')
            else:
                stall_time = _time
        else: 
            stall_time = _time;
        
    
    #At this point, GSTALLING is always false, mean the buffer is streaming
    elif GBUFFER == []:
        #try to consume an empty buffer
        GSTALLING = True
        stall_time = _time
    else:
        #try to consume non empty buffer
        stall_time = 0
        d = GBUFFER[0][2] - GTIME
        if d > 0.01:#some overflow error is compensated
            raise Exception("No content available in buffer, GBUFFER[0][2] > GTIME")
        #time until available buffer is stall time
        #stall_time = min(d, _time)
        #if stall_time == _time:
        #    #then it means none of buffer is consume,
        #    GSTALLING = True
        
        #assume GTIME overlap with timestamp of buffer
        while _time > 0:
            #if there is nothing to consume, and _time required for consumd is > 0
            #then 
            if GBUFFER == []: 
                GSTALLING = True
                break
            _time0 = _time
            _time = _time - GBUFFER[0][1]#index 1 is duration
            #this case mean _time is consumed, good
            if _time < 0:
                GBUFFER[0][1] -= _time0#only reduce duration of first element, GBUFFER remain
                GBUFFER[0][2] += _time0
            
            #this case means _time and buffer is consume, well..
            elif _time == 0:
                GBUFFER = GBUFFER[1:]#may result in [] after this operation is len(GBUFFER)==1
            
            #this case means buffer is consumes but _time > 0, stall!
            elif _time > 0:
                GBUFFER = GBUFFER[1:]#may result in [], continue with the loop
        
        stall_time = _time
        if stall_time < 0: 
            stall_time = 0
        
    if stall_time > 0:
            global GSTALL_LOG
            GSTALL_LOG.append([GTIME, stall_time])
        
    return stall_time


class BlackRegionException(Exception):
    pass



def buffer_consume_blackstall(_iid, _time):
    #consume global buffer based on time passed
    #if a buffer 
    #updated global variable GBUFFER content, GSTALL_LIST flag
    #return stall_time, which is the time system will be stall
    
    global psnr_tilemap_log
    global GTIME
    global GSTALLING
    global GBUFFER
    stall_time = 0
    
    clean_buffer(GTIME)
    

    if GSTALLING == True:
        #is stalling mean empty buffer.
        if GBUFFER != []: 
            temp = [item[1] for item in GBUFFER]
            if sum(temp) > tracesim_header.base_buffer_time:
                raise Exception ('Why there is this case in here, why it is stalling when there is more buffer than required?')
            else:
                stall_time = _time
        else: 
            stall_time = _time;
            
        #add some item indicate that user is seeing black
        #for t in np.arange(0, stall_time, tracesim_header.timestep):
        psnr_tilemap_log.append([_iid, (GTIME)/1000.0, -1, -1])
        
        GTIME += 0;
        
    
    #At this point, GSTALLING is always false, mean the buffer is streaming
    elif GBUFFER == []:
        #try to consume an empty buffer
        GSTALLING = True
        stall_time = _time
        GTIME += 0;
    else:
        #try to consume non empty buffer
        stall_time = 0
        d = GBUFFER[0][2] - GTIME
        if d > 0.01:#some overflow error is compensated
            raise Exception("No content available in buffer, GBUFFER[0][2] > GTIME")

        #assume GTIME overlap with timestamp of buffer
        while _time > 0:
            #if there is nothing to consume, and _time required for consumd is > 0
            #then 
            if GBUFFER == []: 
                GSTALLING = True
                break
            
            _time0 = _time
            #ratio, t0, tpd0, tpd1 = predictor_compare(_iid, _time + GBUFFER[0][2], GBUFFER[0][0])
            ratio, t0, tpd0, tpd1, tile_map_gt = predictor_compare(_iid, GBUFFER[0][2]/1000.0, GBUFFER[0][0])
            #dump downloaded tilemap in buffer, gt tilemap, and the corresponding time
            psnr_tilemap_log.append([_iid, GBUFFER[0][2]/1000.0, GBUFFER[0][0], tile_map_gt])
            logging.debug('BUFFER-CONSUME: _time: {}, GBUFFER[0][1]: {}, \
                          ratio: {}, t0: {}, tpd0: {}, tpd1: {}'.format(_time, GBUFFER[0][1], ratio, t0, tpd0, tpd1))
    
            if ratio < .8: 
                logging.info('User looking at black region, stall now')
                #GSTALLING = True
                #GBUFFER = []#empty buffer, stall, raise error, force the simulator to resend
                if _time > 0:
                    global GSTALL_LOG
                    GSTALL_LOG.append([GTIME, _time])
                raise BlackRegionException("User seeing black region")
            else:
                d = (tpd1 - t0) * 1000
                _time -= d #consume just a little bit buffer here
                GBUFFER[0][1] -= min(d, _time) #buffer also consume 
                GBUFFER[0][2] += min(d, _time) #move on
                
                if GBUFFER[0][1] <= 0: #first elem of buffer is empty now
                    GBUFFER = GBUFFER[1:]
                
                GTIME += min(d, _time)
                
        
        stall_time = _time
        if stall_time < 0: 
            stall_time = 0
        
        logging.debug('BUFFER-CONSUME-AFTER: _time: {}, stall_time: {}, GBUFFER[0][1]: {}, GBUFFER[0][2]: {}'.format\
                      (_time, stall_time, GBUFFER[0][1], GBUFFER[0][2]))
        
    return stall_time


#what a wierd title I have
def eat_frame_with_headpos():
    #TODO: consume buffer frame by frame based on tiles and groundtruth headpos
    #input timestamp normalized
    #tilemap along with this timestamp
    #tilemap in GBUFFER with timestamp
    
    #output: GBUFFER is modified ,
    return


def buffer_fillup(_tile_map, _chunk_duration, _chunk_timestamp):
    global GBUFFER
    global GSTALLING
    _chunk_timestamp = int(_chunk_timestamp)

    GBUFFER.append([_tile_map, _chunk_duration, _chunk_timestamp])
    #GBUFFER.append([None, _chunk_duration, _chunk_timestamp])
    temp = [item[1] for item in GBUFFER]
    if GSTALLING == True and sum(temp) > tracesim_header.base_buffer_time:
        GSTALLING = False
    return


def predictor_loaddata(_iid):
    global H
    global W
    global GPREDICTOR_DATA

    #prediction = test_result[_iid][0][_time_index].reshape(H, W)
    #prediction = centertile_to_fov(salient_to_centertile(prediction, H, W))
    #groundtruth = test_result[_iid][1][_time_index].reshape(H, W)
    #groundtruth = centertile_to_fov(salient_to_centertile(groundtruth, H, W))
    #predictor_timestamp = test_result[_iid][2]
    #predictor_timestamp = (np.array(predictor_timestamp) - predictor_timestamp[0]) + tracesim_header.step_ahead
    #Xtest = test_result[_iid][3]
    
    if _iid in GPREDICTOR_DATA:
        return GPREDICTOR_DATA[_iid]
    
    
    prediction_list = test_result[_iid][0].reshape(-1, H, W)
    prediction_list = np.array([centertile_to_fov(salient_to_centertile(item, H, W)) for item in prediction_list])
    groundtruth_list = test_result[_iid][1].reshape(-1, H, W)
    groundtruth_list = np.array([centertile_to_fov(salient_to_centertile(item, H, W)) for item in groundtruth_list])
    predictor_timestamp = np.array(test_result[_iid][2])
    predictor_timestamp = (predictor_timestamp - predictor_timestamp.min()) + tracesim_header.step_ahead
    predictor_timestamp[:predictor_timestamp.argmin()] = -1
    Xtest = test_result[_iid][3]
    
    GPREDICTOR_DATA[_iid] = prediction_list, groundtruth_list, predictor_timestamp, Xtest
    
    return GPREDICTOR_DATA[_iid]


def predictor_time_to_timeindex(_iid, _time, _predictor_timestamp):   
    result = -1
    for idx, item in enumerate(_predictor_timestamp):
        if np.abs(item - _time) <= tracesim_header.timestep * .7:
            result = idx
            break
    if result > -1:
        return result
    else:
        raise Exception("cannot find time index for IID: {}, TIME: {}, PD TIMESTAMP: {}".format\
                       (_iid, _time, _predictor_timestamp))
            

def predictor_predict(_iid, _time):
    #give the tile predict in next 0.5 seconds given the time.
    #return dummy data first, then will compare later
    global H
    global W
    
    #time_index is the time in future
    prediction_list, groundtruth_list, predictor_timestamp, Xtest = predictor_loaddata(_iid)
    time_index = predictor_time_to_timeindex(_iid, _time, predictor_timestamp)
    prediction = prediction_list[time_index]
    groundtruth = groundtruth_list[time_index]
    
    prediction2 = expand_fov(prediction, _ex=1);
    prediction3 = expand_fov(prediction, _ex=2);
    prediction4 = np.ones(shape=(H, W));
    
    salient = baseline1_current_salient(Xtest, time_index)
    headpos = baseline2_current_headpos(Xtest, time_index)

    bl_cur_sal = centertile_to_fov(salient_to_centertile(salient, H, W))
    bl_cur_pos = centertile_to_fov(headpos)

    return prediction, bl_cur_pos, bl_cur_sal, groundtruth, prediction2, prediction3, prediction4
    
    #return np.array([[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]])

    
def predictor_compare(_iid, _time, _buffer_tile):#the _time get from the top of buffer
    #TODO: given input timestamp, compare groundtruth with the current entry in GBUFFER
    #if it is incompatible (to many black tiles (how many?)), raise error
    #assume _time_index is the 
    
    prediction_list, groundtruth_list, predictor_timestamp, Xtest = predictor_loaddata(_iid)
    time_index = predictor_time_to_timeindex(_iid, _time, predictor_timestamp)
    prediction = prediction_list[time_index] #
    groundtruth = groundtruth_list[time_index] #now what user are looking at
    
    #goal: compare buffer tile with groundtruth tile 
    #overlap region between groundtruth and _buffer_tile
    a = ((groundtruth==1)==_buffer_tile).sum()#number of tile overlap
    b = (groundtruth==1).sum()#number of tile user see (always 36)
    
    return a*1.0/b, _time, predictor_timestamp[time_index], predictor_timestamp[time_index+1], groundtruth


def wait(_iid):
    global GTIME
    
    st = buffer_consume(_iid, tracesim_header.wait_consume_time)
    #GTIME += tracesim_header.wait_consume_time - st
    logging.debug('waited, waited consume time: {}, stall time: {}'.format(tracesim_header.wait_consume_time, st))
    

def ask_server(_iid, _tile_map, _tile_timestamp ):
    #TODO :after asking the server, expect the time 
    #input: _tile_map: requested tile
    #tile_time: timestamp of the tile
    
    #FIRST, SEND to server the tile
    global GTIME

    st = buffer_consume(_iid, tracesim_header.net_delay)
    #GTIME += tracesim_header.net_delay - st
    
    logging.debug('asked server')
    return


def receive_server(_iid, _tile_map, _tile_timestamp, _topic_idx, _tile_map_pos=0, _tile_map_sal=0):
    
    #TODO:
    global GTIME
    chunk_duration = tracesim_header.chunk_duration
    
    
    tilesize_map = server_get_tile_size(_topic_idx, _tile_timestamp, _tile_map)
    tilesize_map_pos = server_get_tile_size(_topic_idx, _tile_timestamp, _tile_map_pos)
    tilesize_map_sal = server_get_tile_size(_topic_idx, _tile_timestamp, _tile_map_sal)
    total_tilesize = tilesize_map.sum()#in byte
    total_tilesize_pos = tilesize_map_pos.sum()
    total_tilesize_sal = tilesize_map_sal.sum()
    
    transfer_time = total_tilesize * 8 * 1.0 / tracesim_header.bandwidth * 1000#convert seconds to milliseconds
    
    #transfer_time = 40
    #will remove gstalling flag and continue playback if it is still
    #buffer consume will consume the buffer until it is empty, then expand the stall log

    st = buffer_consume(_iid, tracesim_header.net_delay)
    #GTIME += tracesim_header.net_delay - st#real consume time
    buffer_fillup(_tile_map, chunk_duration, _tile_timestamp)
    st = buffer_consume(_iid, transfer_time)
    #GTIME += transfer_time - st
    
    logging.debug('received from server TILE_SIZE: {} {} {}'.format(total_tilesize, total_tilesize_pos, total_tilesize_sal))
    return


In [ ]:
import tracesim_header
reload(tracesim_header)


<module 'tracesim_header' from 'tracesim_header.pyc'>

In [14]:
#code the simulation here
#STEP 1: check buffer #need? buffer will automatically drained by itself
#STEP 2: ask predictor for tile
#what is the global?
import tracesim_header

iid = 2

del logging
import logging
reload(logging)

#logging.getLogger().setLevel(logging.DEBUG)
#for hdlr in logging.getLogger().handlers[:]:  # remove all old handlers
#        logging.getLogger().removeHandler(hdlr)
#logging.getLogger().addHandler(logging.FileHandler('./trace_simulation_data/iid{}.log'.format(iid), 'w'))



W, H = 16, 9
GPREDICTOR_DATA = {}
GTIME = 12000.0#in miliseconds, simulator must increase this manually
GBASE_BUFFER = tracesim_header.base_buffer_time#minimum time in buffer before streaming
GSTALLING = True#a flag telling buffer to not drain
GSTALL_LOG = []#a list of [Gtime, stall_duration] elements
GBUFFER = []
BTIME = GTIME#buffer limit time. Condition: BTIME - GTIME <= tracesim_header.base_buffer_time


time_index = int(GTIME*12/1000.0)

topic_idx, user_idx = order_test_list[iid]

##########funcode########CUSTOMECODE
#output 20 users to log file
#read log file, collect the total wait time and total 
buffer_consume = buffer_consume_blackstall
logging.getLogger().setLevel(logging.INFO)

############################
#
while GTIME < 30000:
    logging.info('GTIME: {}'.format(GTIME))
    logging.info('BTIME: {}'.format(BTIME))
    logging.info('GBUFFER: {}'.format(repr(np.array(GBUFFER)[:, 1:].tolist()) if len(GBUFFER) > 0 else GBUFFER))
    #logging.info('GBUFFER: {}'.format(GBUFFER))
    logging.info('GSTALLING FLAG: {}'.format(GSTALLING))
    logging.info('-----------')
    if BTIME - GTIME >= tracesim_header.base_buffer_time:
        GSTALLING = False
        try:
            wait(iid)
        except BlackRegionException:
            BTIME = GTIME
            GBUFFER = []
            GSTALLING = True    
    else:
        try:
            if BTIME - GTIME > tracesim_header.step_ahead * 1000:
                logging.info('PREDICTION: yes')
                tile_map, tile_map_pos, tile_map_sal, gt, tm2, tm3, tm4 = predictor_predict(iid, (BTIME + .5)/1000.0)
                #tile_map = tile_map_sal
                #tile_map = tile_map_pos
            else: 
                logging.info('PREDICTION: no')
                tile_map, tile_map_pos, tile_map_sal, gt, tm2, tm3, tm4 = predictor_predict(iid, (BTIME + .5)/1000.0)
                tile_map = gt
            ask_server(iid, tile_map, BTIME + .5)
            receive_server(iid, tile_map, BTIME, topic_idx, tile_map_pos, tile_map_sal)
            BTIME += 1000.0
        except BlackRegionException:
            BTIME = GTIME
            GBUFFER = []
            GSTALLING = True            
    
        

INFO:root:GTIME: 12000.0
INFO:root:BTIME: 12000.0
INFO:root:GBUFFER: []
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:PREDICTION: no
INFO:root:GTIME: 12000.0
INFO:root:BTIME: 13000.0
INFO:root:GBUFFER: [[1000.0, 12000]]
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:GTIME: 12109.0
INFO:root:BTIME: 13000.0
INFO:root:GBUFFER: [[891.0000000000103, 12108.99999999999]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 12119.78368
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[880.2163200004535, 12119.78367999955], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12248.26944
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[751.7305600036094, 12248.269439996393], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12331.15552
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[668.8444800288478, 12331.155519971155], [1000.0, 13000]]
INFO:root:GSTALLI

INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16319.2417177
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[598.7582823216003, 16319.241717678398], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16419.725153
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[498.274846964806, 16419.72515303519], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16532.1754591
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[385.8245408944329, 16532.175459105565], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16614.4036728
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[303.5963271554623, 16614.403672844535], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16738.2110185
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[179.78898146640194, 16738.211018533595], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False


INFO:root:GTIME: 20871.7399542
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[46.2600458079466, 20871.739954192064], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20988.9196335
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[990.0803664635284, 20927.919633536472]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 21014.0019552
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[903.9980447641751, 21014.001955235824], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21117.0058657
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[800.9941342925279, 21117.005865707473], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21237.0175971
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[680.9824028775749, 21237.017597122427], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21345

INFO:root:GBUFFER: [[221.02652221466056, 24886.973477785337], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24986.9204334
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[121.07956664399734, 24986.920433356], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25094.6817334
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[1029.2386999320072, 25078.761300067992]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25201.4538674
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[906.5461326080391, 25201.453867391963]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 25190.0124692
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[917.9875307586451, 25190.012469241363], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25266.0374077
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[841.9625922759083

INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29103.9888704
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[4.011129635684455, 29103.98887036432], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29199.9666111
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[969.0333889070558, 29138.966611092943]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 29220.1403066
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[887.8596934422951, 29220.14030655771], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29355.4209197
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[752.5790803268706, 29355.42091967314], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29446.262759
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[661.7372409805832, 29446.262759019428], [1000.0, 30108]]
INFO:root:GSTALLING FLAG:

In [ ]:
#code the simulation here
#STEP 1: check buffer #need? buffer will automatically drained by itself
#STEP 2: ask predictor for tile
#what is the global?
import tracesim_header

#raise #I dont want simulation now


iid = 3


##########funcode########CUSTOMECODE
#output 20 users to log file
#read log file, collect the total wait time and total 
buffer_consume = buffer_consume_blackstall
psnr_filetoken = "sal"#####IMPORTANT, DON'T FORGET THIS CONFIG LINE: pred, sal

############################
for iid in np.arange(23, 42):
    psnr_tilemap_log = []#####this is to log the psnr quality, empty it every loops
    
    #logging.basicConfig(filename='./trace_simulation_data/iid{}.log'.format(iid))
    logging.getLogger().setLevel(logging.DEBUG)
    for hdlr in logging.getLogger().handlers[:]:  # remove all old handlers
        logging.getLogger().removeHandler(hdlr)
    logging.getLogger().addHandler(logging.FileHandler('./trace_simulation_data/iid{}_tm2.log'.format(iid), 'w'))
    
    W, H = 16, 9
    GPREDICTOR_DATA = {}
    GTIME = 12000.0#in miliseconds, simulator must increase this manually
    GBASE_BUFFER = tracesim_header.base_buffer_time#minimum time in buffer before streaming
    GSTALLING = True#a flag telling buffer to not drain
    GSTALL_LOG = []#a list of [Gtime, stall_duration] elements
    GBUFFER = []
    BTIME = GTIME#buffer limit time. Condition: BTIME - GTIME <= tracesim_header.base_buffer_time
    time_index = int(GTIME*12/1000.0)
    topic_idx, user_idx = order_test_list[iid]
    
    if topic_idx == 5: continue

    while GTIME < 28000:
        logging.info('GTIME: {}'.format(GTIME))
        logging.info('BTIME: {}'.format(BTIME))
        logging.info('GBUFFER: {}'.format(repr(np.array(GBUFFER)[:, 1:].tolist()) if len(GBUFFER) > 0 else GBUFFER))
        #logging.info('GBUFFER: {}'.format(GBUFFER))
        logging.info('GSTALLING FLAG: {}'.format(GSTALLING))
        logging.info('-----------')
        if BTIME - GTIME >= tracesim_header.base_buffer_time:
            GSTALLING = False
            try:
                wait(iid)
            except BlackRegionException:
                BTIME = GTIME
                GBUFFER = []
                GSTALLING = True    
        else:
            try:
                if BTIME - GTIME > tracesim_header.step_ahead * 1000:
                    logging.info('PREDICTION: yes')
                    tile_map, tile_map_pos, tile_map_sal, gt, tm2, tm3, tm4 = predictor_predict(iid, (BTIME + .5)/1000.0)
                    #tile_map = tm2#this code try wider windows for our model
                    tile_map = tile_map_sal#this code try the baseline
                    #tile_map = tile_map_pos
                    #tile_map = tm4
                else: 
                    logging.info('PREDICTION: no')
                    tile_map, tile_map_pos, tile_map_sal, gt, tm2, tm3, tm4 = predictor_predict(iid, (BTIME + .5)/1000.0)
                    tile_map = gt
                ask_server(iid, tile_map, BTIME + .5)
                receive_server(iid, tile_map, BTIME, topic_idx, tile_map_pos, tile_map_sal)
                BTIME += 1000.0
            except BlackRegionException:
                BTIME = GTIME
                GBUFFER = []
                GSTALLING = True      
                
                
    fiid = iid
    viid, uiid = order_test_list[fiid]
    import pickle
    pickle.dump(psnr_tilemap_log, open('./psnr_data/psnr_gttilemap_viid{}_uiid{}_{}_log'.format(viid, uiid, psnr_filetoken), 'wb'))

In [24]:
#what are existing problem
#without stalling, 3 baseline, what is the 
#stalling when user see black region: not yet, will do it later
#

#right now I need result for bandwidth consumption withou stalling of 10 pairs
#how to integrate specific videos (video data ok), head position not ok, need 3 models
#how to extract head position based on model?

In [ ]:
#good case: 13.91


iid = 8
tile_map, tile_map_pos, tile_map_sal, gt, _ , _ , _ = predictor_predict(iid, 11.11)
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

tile_map_v2 = expand_fov(tile_map, _r=3, _ex=1)

#tile_map, tile_map_pos, tile_map_sal, gt

plt.imshow(tile_map_sal)
plt.title('salient')
plt.figure()
plt.imshow(tile_map_pos)
plt.title('cur pos')
plt.figure()
plt.imshow(gt)
plt.title('ground truth')
plt.figure()
plt.imshow(tile_map)
plt.title('prediction')
plt.figure()
plt.imshow(tile_map_v2)
plt.title('prediction v2')

In [ ]:
#unit test code
H = 9
W = 16

#include inside lstm3_eval
temp = salient_to_centertile(test_result[0][0][0], H, W)
tile_map = centertile_to_fov(temp)
    `
GBUFFER = [[None, 4]]#should have temp, but now for simplicity
GTIME = 0
GSTALLING = False
GSTALL_LIST = []
print GBUFFER
print GSTALLING
print buffer_consume(2);
print GBUFFER
print GSTALLING
print buffer_consume(3);
print GBUFFER
print GSTALLING
print buffer_consume(4)
print GBUFFER
print GSTALLING

In [ ]:
test_result[iid][0].shape

In [ ]:
#parse log, plot the buffer over time
text = '''INFO:root:GTIME: 12000.0
INFO:root:BTIME: 12000.0
INFO:root:GBUFFER: []
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:PREDICTION: no
INFO:root:GTIME: 12000.0
INFO:root:BTIME: 13000.0
INFO:root:GBUFFER: [[1000.0, 12000]]
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:GTIME: 12109.0
INFO:root:BTIME: 13000.0
INFO:root:GBUFFER: [[891.0000000000103, 12108.99999999999]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 12119.78368
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[880.2163200004535, 12119.78367999955], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12248.26944
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[751.7305600036094, 12248.269439996393], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12331.15552
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[668.8444800288478, 12331.155519971155], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12456.4665599
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[543.5334400865339, 12456.46655991347], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12580.3996797
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[419.6003202595849, 12580.39967974042], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12700.1990392
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[299.8009607787504, 12700.199039221256], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12807.5971177
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[192.4028823362397, 12807.597117663767], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 12940.791353
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[59.20864700870702, 12940.7913529913], [1000.0, 13000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13077.791353
INFO:root:BTIME: 14000.0
INFO:root:GBUFFER: [[963.2086470087017, 13036.791352991298]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 13057.7507246
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[942.2492753915761, 13057.750724608426], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13187.2521738
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[812.7478261747212, 13187.25217382528], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13323.7565215
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[676.2434785241657, 13323.756521475834], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13438.0521718
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[561.9478281933394, 13438.052171806661], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13572.1565154
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[427.84348458001455, 13572.156515419985], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13661.2521234
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[338.74787664012695, 13661.252123359873], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13737.7563701
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[262.2436299203894, 13737.756370079609], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13841.2691102
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[158.73088976117558, 13841.269110238822], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 13962.8073307
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[37.19266928353164, 13962.807330716465], [1000.0, 14000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14040.4219921
INFO:root:BTIME: 15000.0
INFO:root:GBUFFER: [[1008.578007850604, 13991.421992149397]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 14073.6284587
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[926.3715413253863, 14073.628458674615], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14208.0276694
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[791.9723306030982, 14208.027669396904], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14326.0553388
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[673.9446612061894, 14326.055338793813], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14400.4427104
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[599.55728964953, 14400.44271035047], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14506.3281311
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[493.6718689485836, 14506.328131051416], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14634.9843932
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[365.0156068457633, 14634.984393154235], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 14768.9531795
INFO:root:BTIME: 16000.0
INFO:root:GBUFFER: [[231.0468205372917, 14768.953179462707], [1000.0, 15000]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:User looking at black region, stall now
INFO:root:GTIME: 14918.8595384
INFO:root:BTIME: 14918.8595384
INFO:root:GBUFFER: []
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:PREDICTION: no
INFO:root:GTIME: 14918.8595384
INFO:root:BTIME: 15918.8595384
INFO:root:GBUFFER: [[999.1404616118725, 14918.859538388127]]
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:GTIME: 15053.5786152
INFO:root:BTIME: 15918.8595384
INFO:root:GBUFFER: [[864.4213848356212, 15053.578615164379]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 15043.7374028
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[874.2625972199065, 15043.737402780092], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15176.2122083
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[741.7877916597175, 15176.212208340283], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15258.636625
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[659.3633749791526, 15258.636625020847], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15379.9098751
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[538.0901249374639, 15379.909875062536], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15491.7296252
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[426.2703748123922, 15491.729625187607], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15575.1888756
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[342.8111244371774, 15575.18887556282], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15699.5666267
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[218.43337331153703, 15699.56662668846], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15820.6998801
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[97.300119934614, 15820.699880065382], [1000.0, 15918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 15957.6998801
INFO:root:BTIME: 16918.8595384
INFO:root:GBUFFER: [[981.3001199346207, 15936.699880065378]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 16029.4478016
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[888.5521983655746, 16029.447801634426], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16120.5824131
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[797.4175869246227, 16120.582413075379], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16201.7472392
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[716.2527607738634, 16201.747239226137], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16319.2417177
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[598.7582823216003, 16319.241717678398], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16419.725153
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[498.274846964806, 16419.72515303519], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16532.1754591
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[385.8245408944329, 16532.175459105565], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16614.4036728
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[303.5963271554623, 16614.403672844535], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16738.2110185
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[179.78898146640194, 16738.211018533595], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16857.6330556
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[60.36694439920129, 16857.6330556008], [1000.0, 16918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 16968.8991668
INFO:root:BTIME: 17918.8595384
INFO:root:GBUFFER: [[978.1008331976068, 16939.899166802395]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 17048.435514
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[869.5644859879734, 17048.435514012028], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17158.306542
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[759.6934579639195, 17158.30654203608], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17276.4523363
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[641.5476637113726, 17276.45233628863], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17401.3570089
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[516.6429911341017, 17401.357008865903], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17524.0710266
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[393.9289734022983, 17524.07102659771], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17640.2130798
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[277.7869202068613, 17640.21307979315], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17736.6392394
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[181.36076062056267, 17736.639239379452], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17836.9177181
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[81.08228186164132, 17836.91771813837], [1000.0, 17918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 17973.9177181
INFO:root:BTIME: 18918.8595384
INFO:root:GBUFFER: [[981.0822818616309, 17936.91771813837]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 18052.2336735
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[865.7663265404864, 18052.23367345952], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18153.7010204
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[764.298979621455, 18153.701020378554], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18269.1030611
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[648.8969388643534, 18269.10306113566], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18363.3091834
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[554.6908165930178, 18363.309183406993], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18474.4734673
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[443.52653274407965, 18474.473467255935], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18601.4204018
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[316.57959823219323, 18601.42040176782], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18730.2612053
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[187.73879469656416, 18730.26120530345], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 18864.7836159
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[53.2163840896751, 18864.78361591034], [1000.0, 18918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19001.7836159
INFO:root:BTIME: 19918.8595384
INFO:root:GBUFFER: [[961.2163840896628, 18956.783615910335]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 19010.9179832
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[907.0820168069548, 19010.917983193045], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19139.7539496
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[778.2460504208831, 19139.753949579117], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19274.2618487
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[643.7381512626625, 19274.261848737337], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19362.7855462
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[555.2144537879943, 19362.785546212006], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19477.2843697
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[440.71563030398306, 19477.284369696023], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19593.8531091
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[324.14689091194873, 19593.853109088057], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19691.5593273
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[226.44067273581896, 19691.559327264185], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19795.6779818
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[122.32201820745743, 19795.67798179255], [1000.0, 19918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19904.3559636
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[1028.3220182074501, 19889.67798179255]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 19993.0678908
INFO:root:BTIME: 20918.8595384
INFO:root:GBUFFER: [[924.9321092447012, 19993.067890755297]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 20073.1418538
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[844.8581462235541, 20073.141853776444], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20184.4255613
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[733.5744386706667, 20184.425561329335], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20317.4044906
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[600.5955093653017, 20317.4044906347], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20433.8089813
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[484.1910187305965, 20433.80898126941], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20548.4718502
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[369.52814984475015, 20548.471850155256], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20665.4155505
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[252.58444953423069, 20665.41555046578], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20764.2466514
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[153.75334860265627, 20764.246651397356], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20871.7399542
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[46.2600458079466, 20871.739954192064], [1000.0, 20918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 20988.9196335
INFO:root:BTIME: 21918.8595384
INFO:root:GBUFFER: [[990.0803664635284, 20927.919633536472]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 21014.0019552
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[903.9980447641751, 21014.001955235824], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21117.0058657
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[800.9941342925279, 21117.005865707473], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21237.0175971
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[680.9824028775749, 21237.017597122427], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21345.0527914
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[572.9472086327035, 21345.0527913673], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21480.1583741
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[437.84162589810614, 21480.158374101895], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21570.4751223
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[347.5248776943111, 21570.475122305692], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21703.8009784
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[214.1990215544858, 21703.80097844552], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21820.6019569
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[97.39804310896997, 21820.601956891038], [1000.0, 21918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 21937.8058707
INFO:root:BTIME: 22918.8595384
INFO:root:GBUFFER: [[986.1941293268849, 21931.805870673117]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 21971.6803867
INFO:root:BTIME: 23918.8595384
INFO:root:GBUFFER: [[946.3196132688106, 21971.680386731194], [1000.0, 22918]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:User looking at black region, stall now
INFO:root:GTIME: 22108.6803867
INFO:root:BTIME: 22108.6803867
INFO:root:GBUFFER: []
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:PREDICTION: no
INFO:root:GTIME: 22108.6803867
INFO:root:BTIME: 23108.6803867
INFO:root:GBUFFER: [[999.3196132688099, 22108.68038673119]]
INFO:root:GSTALLING FLAG: True
INFO:root:-----------
INFO:root:GTIME: 22196.0411602
INFO:root:BTIME: 23108.6803867
INFO:root:GBUFFER: [[911.9588398064332, 22196.041160193567]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 22234.4886048
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[873.5113951610099, 22234.488604838993], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22321.4658145
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[786.5341854830258, 22321.465814516978], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22456.3974436
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[651.6025564490707, 22456.397443550937], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22554.1795484
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[553.8204515925198, 22554.17954840749], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22650.5386452
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[457.46135477752887, 22650.53864522248], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22750.6159357
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[357.38406433258046, 22750.61593566743], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22861.847807
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[246.15219299770627, 22861.847807002305], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 22943.543421
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[164.4565789930996, 22943.54342100691], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23062.630263
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[45.36973697926595, 23062.630263020743], [1000.0, 23108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23178.8907891
INFO:root:BTIME: 24108.6803867
INFO:root:GBUFFER: [[942.109210937771, 23165.89078906223]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 23196.7504466
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[911.2495534443933, 23196.75044655561], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23318.2513397
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[789.7486603331811, 23318.251339666822], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23430.754019
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[677.245980999521, 23430.754019000484], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23516.262057
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[591.7379429985564, 23516.262057001448], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23646.786171
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[461.2138289956488, 23646.786171004354], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23761.289368
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[346.7106319651667, 23761.28936803484], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23877.8681041
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[230.1318958954991, 23877.868104104506], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 23975.6043123
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[132.39568768647317, 23975.604312313535], [1000.0, 24108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24096.2086246
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[1016.3956876864681, 24091.60431231353]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24206.668997
INFO:root:BTIME: 25108.6803867
INFO:root:GBUFFER: [[901.3310029834377, 24206.66899701656]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 24179.6436454
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[928.3563545860396, 24179.64364541396], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24313.9309362
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[794.0690637581355, 24313.93093624186], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24401.7928087
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[706.2071912744238, 24401.79280872557], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24509.3424698
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[598.6575301954306, 24509.342469804564], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24610.0274094
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[497.9725905863087, 24610.027409413688], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24723.0822282
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[384.91777175893515, 24723.08222824106], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24811.6578259
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[296.3421740715556, 24811.65782592844], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24886.9734778
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[221.02652221466056, 24886.973477785337], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 24986.9204334
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[121.07956664399734, 24986.920433356], [1000.0, 25108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25094.6817334
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[1029.2386999320072, 25078.761300067992]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25201.4538674
INFO:root:BTIME: 26108.6803867
INFO:root:GBUFFER: [[906.5461326080391, 25201.453867391963]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 25190.0124692
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[917.9875307586451, 25190.012469241363], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25266.0374077
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[841.9625922759083, 25266.0374077241], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25368.1122232
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[739.8877768277197, 25368.112223172287], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25485.3366695
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[622.6633304831461, 25485.33666951686], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25585.0100086
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[522.9899914494412, 25585.010008550562], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25695.0300257
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[412.969974348301, 25695.030025651704], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25773.090077
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[334.90992304489293, 25773.090076955115], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 25881.2702309
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[226.72976913464794, 25881.270230865364], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26016.8106926
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[91.18930740392642, 26016.81069259609], [1000.0, 26108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26149.4320778
INFO:root:BTIME: 27108.6803867
INFO:root:GBUFFER: [[995.5679222117325, 26112.432077788268]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 26154.7262035
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[953.2737964668087, 26154.72620353319], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26270.1786106
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[837.8213894004041, 26270.1786105996], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26364.5358318
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[743.4641682012134, 26364.535831798792], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26477.2866544
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[630.7133456096216, 26477.286654390384], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26607.8599632
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[500.14003682886624, 26607.85996317114], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26722.8797054
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[385.12029463091284, 26722.879705369094], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26840.6391161
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[267.36088389270583, 26840.6391161073], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 26941.9173483
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[166.08265167811487, 26941.917348321895], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27056.752045
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[51.24795503430582, 27056.752044965706], [1000.0, 27108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27193.752045
INFO:root:BTIME: 28108.6803867
INFO:root:GBUFFER: [[959.2479550343029, 27148.752044965695]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 27144.5577724
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[963.4422275724346, 27144.557772427564], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27223.6733173
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[884.3266827173159, 27223.673317282683], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27335.0199518
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[772.9800481519674, 27335.01995184803], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27417.0598555
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[690.9401444559198, 27417.059855544074], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27537.1795666
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[570.8204333677818, 27537.179566632214], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27645.5386999
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[462.46130010335855, 27645.53869989664], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27781.6160997
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[326.3839003101024, 27781.616099689894], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27874.8482991
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[233.15170093032384, 27874.848299069676], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 27975.7863926
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[132.21360744253423, 27975.786392557467], [1000.0, 28108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28074.5727851
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[1038.2136074425282, 28069.78639255747]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28186.7183553
INFO:root:BTIME: 29108.6803867
INFO:root:GBUFFER: [[921.2816446551676, 28186.718355344834]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 28211.4630195
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[896.5369805172439, 28211.463019482755], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28345.3890584
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[762.6109415517267, 28345.389058448272], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28432.1671753
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[675.8328246551646, 28432.167175344835], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28566.501526
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[541.498473965493, 28566.501526034506], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28658.0122083
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[449.9877917238966, 28658.012208276105], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28740.0366248
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[367.9633751716974, 28740.036624828303], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28860.1098745
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[247.89012551507562, 28860.109874484926], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 28968.3296235
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[139.67037654523332, 28968.329623454767], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29103.9888704
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[4.011129635684455, 29103.98887036432], [1000.0, 29108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29199.9666111
INFO:root:BTIME: 30108.6803867
INFO:root:GBUFFER: [[969.0333889070558, 29138.966611092943]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:PREDICTION: yes
INFO:root:GTIME: 29220.1403066
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[887.8596934422951, 29220.14030655771], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29355.4209197
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[752.5790803268706, 29355.42091967314], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29446.262759
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[661.7372409805832, 29446.262759019428], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29522.1020722
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[585.8979278445909, 29522.102072155423], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29631.3062165
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[476.6937835337345, 29631.306216466277], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29742.4497317
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[365.55026826980645, 29742.449731730205], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29851.3491952
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[256.6508048094015, 29851.34919519061], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------
INFO:root:GTIME: 29926.0475856
INFO:root:BTIME: 31108.6803867
INFO:root:GBUFFER: [[181.9524144281637, 29926.04758557185], [1000.0, 30108]]
INFO:root:GSTALLING FLAG: False
INFO:root:-----------'''.split('INFO:root:-----------')

In [ ]:
##### what I am doing?
###PARSE THE LOG
#iid = 0
#text = open('./trace_simulation_data/iid{}.log'.format(iid)).read().split('INFO:root:-----------')

if text[0].find('GBUFFER: []') >= 0:
    text = text[1:]
buffer_caplog = []
gtime_log = []
btime_log = []
bw_log = []

for item0 in text:
    if item0.find('GBUFFER: []') >= 0:
        continue
    temp = item0.split('\n')
    for item in temp:
        if item.find(":GTIME:") >= 0:
            gtime_log.append(float(item.split(' ')[-1]))
        elif item.find(":BTIME:") >= 0:
            btime_log.append(float(item.split(' ')[-1]))
        elif item.find(":GBUFFER:") >= 0:
            gbt = eval(item[item.find('[['):])
            buffer_cap = sum([item[1] for item in gbt])
            buffer_caplog.append(buffer_cap)
for tt in text:
    if tt.find('TILE_SIZE:') > 0:
        temp = tt.split('\n')
        bw_item = []
        for item in temp:
            if item.find(":GTIME:") >= 0:
                bw_item.append(float(item.split(' ')[-1]))
            elif item.find('TILE_SIZE:') >= 0:
                bw_item.append(float(item.split(' ')[-3]))
                bw_item.append(float(item.split(' ')[-2]))
                bw_item.append(float(item.split(' ')[-1]))
        bw_log.append(bw_item)
from matplotlib import pyplot as plt
%matplotlib inline

x = gtime_log
y = buffer_caplog
plt.plot(x, y)
plt.scatter(x, y)
plt.ylabel('Buffer capability (milliseconds)')
plt.xlabel('Video time (milliseconds)')
plt.title('Amount of data in seconds inside buffer though video time')
plt.figure()
bw_log = np.array(bw_log)
print bw_log.shape
y = bw_log[:, 0]/1024.0
y_pos = bw_log[:, 1]/1024.0
y_sal = bw_log[:, 2]/1024.0
x = bw_log[:, 3]

plt.plot(x, y, label='LSTM model')
plt.scatter(x, y, )
plt.plot(x, y_pos, label='Current user head position')
plt.scatter(x, y_pos)
plt.plot(x, y_sal, label='Salient map')
plt.scatter(x, y_sal)
plt.title('Data transfer over network through time. Bandwidth 100Mbps')
plt.xlabel('Video time (milliseconds)')
plt.ylabel('Data transfer over network')
plt.legend()
plt.axis([0, 10000, 650, 800])

In [ ]:
buffer_caplog = []
gtime_log = []
btime_log = []
bw_log = []

for item0 in text:
    temp = item0.split('\n')
    for item in temp:
        if item.find(":GTIME:") >= 0:
            gtime_log.append(float(item.split(' ')[-1]))
        elif item.find(":BTIME:") >= 0:
            btime_log.append(float(item.split(' ')[-1]))
        elif item.find(":GBUFFER:") >= 0:
            gbt = eval(item[item.find('[['):])
            buffer_cap = sum([item[1] for item in gbt])
            buffer_caplog.append(buffer_cap)

In [8]:
order_test_list[12]

(7, 36)

#desired result:
criteria: total stop time/ stall time with buffer simulation
          bandwith usage from retransmission
result 1: different window sizes vs black region, retransmission, bandwidth consumption
reuslt 2: with 

In [11]:
#examine the psnr_tilemap_loge, see if anything interesting
len(psnr_tilemap_log)

746

In [12]:
fiid = psnr_tilemap_log[0][0]
viid, uiid = order_test_list[fiid]
print fiid
import pickle
pickle.dump(psnr_tilemap_log, open('./psnr_data/psnr_tilemap_viid{}_uiid{}_pred_log'.format(viid, uiid), 'wb'))

11


(8, 28)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

idx = 3
plt.imshow(psnr_tilemap_log[idx][2])
plt.figure()
plt.imshow(psnr_tilemap_log[idx][3])

In [17]:
viid, uiid

(8, 28)

In [12]:
order_test_list[1:43]

[(5, 38),
 (8, 27),
 (8, 32),
 (5, 12),
 (7, 30),
 (5, 38),
 (8, 2),
 (7, 17),
 (8, 6),
 (8, 0),
 (8, 28),
 (7, 36),
 (8, 18),
 (7, 0),
 (8, 0),
 (8, 23),
 (7, 28),
 (7, 20),
 (6, 13),
 (8, 21),
 (6, 24),
 (5, 29),
 (7, 27),
 (6, 39),
 (6, 0),
 (5, 36),
 (5, 36),
 (5, 13),
 (5, 9),
 (6, 14),
 (6, 14),
 (8, 36),
 (7, 27),
 (6, 19),
 (6, 29),
 (7, 9),
 (6, 33),
 (8, 15),
 (5, 27),
 (6, 0),
 (5, 17),
 (5, 34)]

In [14]:
order_test_list[21]

(6, 24)